In [4]:
! pip install kaggle

In [10]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d kazanova/sentiment140

 80% 65.0M/80.9M [00:00<00:00, 87.1MB/s]
100% 80.9M/80.9M [00:00<00:00, 90.2MB/s]


In [12]:
from zipfile import ZipFile
datasets = '/content/sentiment140.zip'

with ZipFile(datasets, 'r') as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


In [13]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding='windows-1252')

In [30]:
data.head()

,target,id,date,flag,user,text,cleaned_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [39]:
data.isnull().sum()

target          0
id              0
date            0
flag            0
user            0
text            0
cleaned_text    0
dtype: int64

In [3]:
data['target'].value_counts()

NameError: name 'data' is not defined

In [16]:
data.replace({'target': {4: 1}}, inplace=True)

In [17]:
# Function for text preprocessing
def preprocess_text(text):
    stemmer = PorterStemmer()
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [stemmer.stem(word) for word in text if word not in stopwords.words('english')]
    return ' '.join(text)

In [18]:
data['cleaned_text'] = data['text'].apply(preprocess_text)

In [19]:
# Prepare data for LSTM
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['cleaned_text'])

In [20]:
sequences = tokenizer.texts_to_sequences(data['cleaned_text'])
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')


In [22]:
X = padded_sequences
y = data['target'].values

In [24]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [26]:
# Define LSTM model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

In [27]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))


In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2)

Epoch 1/5
8000/8000 [==============================] - 989s 123ms/step - loss: 0.5018 - accuracy: 0.7415 - val_loss: 0.4556 - val_accuracy: 0.7851
Epoch 2/5
8000/8000 [==============================] - 946s 118ms/step - loss: 0.3920 - accuracy: 0.8252 - val_loss: 0.4776 - val_accuracy: 0.7757
Epoch 3/5
8000/8000 [==============================] - 942s 118ms/step - loss: 0.3108 - accuracy: 0.8673 - val_loss: 0.5128 - val_accuracy: 0.7701
Epoch 4/5
8000/8000 [==============================] - 937s 117ms/step - loss: 0.2653 - accuracy: 0.8862 - val_loss: 0.5576 - val_accuracy: 0.7658
Epoch 5/5
8000/8000 [==============================] - 931s 116ms/step - loss: 0.2364 - accuracy: 0.8977 - val_loss: 0.6262 - val_accuracy: 0.7600


In [37]:
# Evaluate LSTM model on training and test data
sample_size = 1000  # Adjust this value based on your dataset size
sample_indices = np.random.choice(len(X_test), size=sample_size, replace=False)

X_test_subset = X_test[sample_indices]
y_test_subset = y_test[sample_indices]

test_acc = model.evaluate(X_test_subset, y_test_subset, verbose=1)[1]
print("Accuracy score of test data (LSTM):", test_acc)

32/32 [==============================] - 1s 18ms/step - loss: 0.6495 - accuracy: 0.7570
Accuracy score of test data (LSTM): 0.7570000290870667


###meaning of verbose : if you want to see and handling the progress of evaluting model

In [36]:
train_acc = model.evaluate(X_train, y_train, verbose=0)[1]
test_acc = model.evaluate(X_test, y_test, verbose=0)[1]


In [38]:
print("Accuracy score of training data (LSTM):", train_acc)
print("Accuracy score of test data (LSTM):", test_acc)

Accuracy score of training data (LSTM): 0.8874617218971252
Accuracy score of test data (LSTM): 0.7570000290870667
